In [2]:
import pandas as pd
import numpy as np
import pyBigWig
k562_positive=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/K562_unt.sort.bed.gz_plus.bw')
k562_negative=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/K562_unt.sort.bed.gz_minus.bw')
h3k27ac_bigwig=pyBigWig.open('/oak/stanford/groups/akundaje/laks/proseq_chromputer/wgEncodeBroadHistoneK562H3k27acStdSig.bigWig')
h3k27ac_peak_replicated_bed=pd.read_csv('/oak/stanford/groups/akundaje/laks/proseq_chromputer/wgEncodeBroadHistoneK562H3k27acStdPk.broadPeak',sep='\t',header=None)

In [ ]:
h3k27ac_peak_replicated_bed['diff']=h3k27ac_peak_replicated_bed[2]-h3k27ac_peak_replicated_bed[1]
h3k27ac_peak_replicated_bed['absolute_diff']=10000-h3k27ac_peak_replicated_bed['diff']
import math
h3k27ac_peak_replicated_bed['left']=h3k27ac_peak_replicated_bed['absolute_diff'].apply(lambda x : math.floor(float(x)/2))
h3k27ac_peak_replicated_bed['right']=h3k27ac_peak_replicated_bed['absolute_diff'].apply(lambda x : math.ceil(float(x)/2))
h3k27ac_peak_replicated_bed['left_new']=h3k27ac_peak_replicated_bed[1]-h3k27ac_peak_replicated_bed['left']
h3k27ac_peak_replicated_bed['right_new']=h3k27ac_peak_replicated_bed[2]+h3k27ac_peak_replicated_bed['right']
h3k27ac_peak_replicated_bed['new_diff']=h3k27ac_peak_replicated_bed['right_new']-h3k27ac_peak_replicated_bed['left_new']
h3k27ac_peak_replicated_bed['left_new']=h3k27ac_peak_replicated_bed['left_new'].astype(int)
h3k27ac_peak_replicated_bed['right_new']=h3k27ac_peak_replicated_bed['right_new'].astype(int)

In [ ]:
h3k27ac_peaks=h3k27ac_peak_replicated_bed[[0,'left_new','right_new']]
h3k27ac_peaks.columns=['chr','start','end']
h3k27ac_peaks_val=h3k27ac_peaks[h3k27ac_peaks['chr']=='chr22']
h3k27ac_peaks_test=h3k27ac_peaks[h3k27ac_peaks['chr']=='chr19']
h3k27ac_peaks=h3k27ac_peaks[h3k27ac_peaks['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr20','chr21','chrX'])]
h3k27ac_peaks['id']=h3k27ac_peaks['chr'].astype(str)+'_'+h3k27ac_peaks['start'].astype(str)+'_'+h3k27ac_peaks['end'].astype(str)

In [ ]:
# TRAINING PREPROCESSING
import statistics

train_list = []
for index, row in h3k27ac_peaks.iterrows():
    start = row['start']
    end = row['end']
    chrom = row['chr']
    mean = (start + end) // 2
    
    bound = k562_positive.chroms(chrom)
    
    # TODO: use try catch instead?
    if start >= 0 and end <= bound:
        if mean + 500 <= bound and mean + 2500 <= bound and mean + 5000 <= bound and mean + 25000 <= bound \
        and mean - 500 >= 0 and mean - 2500 >= 0 and mean - 5000 >= 0 and mean - 25000 >= 0:
            example = {
                '1kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, mean + 500))),
                '5kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, mean + 2500))),
                '10kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, mean + 5000))),
                '50kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, mean + 25000))),
                '1kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, mean + 500))),
                '5kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, mean + 2500))),
                '10kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, mean + 5000))),
                '50kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, mean + 25000))),
                'mean_peak_output': np.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end)))
            }
        else:
            continue
    else:
        continue
    train_list.append(example)
    
bins_train = pd.DataFrame(train_list)
print(bins_train)    

In [ ]:
# VALIDATION AND TEST PREPROCESSING
val_list = []
for index, row in h3k27ac_peaks_val.iterrows():    
    start = row['start'] 
    end = row['end']
    chrom = row['chr']
    mean = (start + end) // 2           

    bound = k562_positive.chroms(chrom)
    
    if start >= 0 and end <= bound:
        if mean + 500 <= bound and mean + 2500 <= bound and mean + 5000 <= bound and mean + 25000 <= bound \
        and mean - 500 >= 0 and mean - 2500 >= 0 and mean - 5000 >= 0 and mean - 25000 >= 0:
            example = {
                '1kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, mean + 500))),
                '5kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, mean + 2500))),
                '10kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, mean + 5000))),
                '50kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, mean + 25000))),
                '1kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, mean + 500))),
                '5kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, mean + 2500))),
                '10kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, mean + 5000))),
                '50kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, mean + 25000))),
                'mean_peak_output': np.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end)))
            }
        else:
            continue
    else:
        continue
    val_list.append(example)
    
bins_val = pd.DataFrame(val_list)
print(bins_val)


test_list = []
for index, row in h3k27ac_peaks_test.iterrows():    
    start = row['start'] 
    end = row['end']
    chrom = row['chr']
    mean = (start + end) // 2

    bound = k562_positive.chroms(chrom)
    
    if start >= 0 and end <= bound:
        if mean + 500 <= bound and mean + 2500 <= bound and mean + 5000 <= bound and mean + 25000 <= bound \
        and mean - 500 >= 0 and mean - 2500 >= 0 and mean - 5000 >= 0 and mean - 25000 >= 0:
            example = {
                '1kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, mean + 500))),
                '5kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, mean + 2500))),
                '10kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, mean + 5000))),
                '50kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, mean + 25000))),
                '1kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, mean + 500))),
                '5kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, mean + 2500))),
                '10kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, mean + 5000))),
                '50kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, mean + 25000))),
                'mean_peak_output': np.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end)))
            }
        else:
            continue
    else:
        continue
    test_list.append(example)
    
bins_test = pd.DataFrame(test_list)
print(bins_test)

In [ ]:
# save the dataframes
bins_train.to_csv(r'/srv/scratch/jjwang01/proseq/bins_train.csv.gz', compression='gzip')
bins_val.to_csv(r'/srv/scratch/jjwang01/proseq/bins_val.csv.gz', compression='gzip')
bins_test.to_csv(r'/srv/scratch/jjwang01/proseq/bins_test.csv.gz', compression='gzip')

In [ ]:
# if dataframes are saved
bins_train = pd.read_csv('/srv/scratch/jjwang01/proseq/bins_train.csv.gz', compression='gzip')
bins_val = pd.read_csv('/srv/scratch/jjwang01/proseq/bins_val.csv.gz', compression='gzip')
bins_test = pd.read_csv('/srv/scratch/jjwang01/proseq/bins_test.csv.gz', compression='gzip')

In [ ]:
from sklearn.linear_model import LinearRegression
y_train = bins_train['mean_peak_output'].to_numpy()
X_train = bins_train.drop(['mean_peak_output'], axis=1).to_numpy()
y_val = bins_val['mean_peak_output'].to_numpy()
X_val = bins_val.drop(['mean_peak_output'], axis=1).to_numpy()
y_test = bins_test['mean_peak_output'].to_numpy()
X_test = bins_test.drop(['mean_peak_output'], axis=1).to_numpy()

reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.score(X_val, y_val))
print(reg.score(X_test, y_test))

In [ ]:
"""
# DO NOT RUN
# uses bound function, rather than excluding examples
# PREPROCESSING
# for each different peak listed as a row in h3k27ac_peak_replicated_bed, match those regions to the proseq
# positive and negative strand values
import statistics

def preprocess():
    y_train = []
    def bound(mean, n, positive, chrom):
        if k562_positive.chroms(chrom) < mean + n:
            return mean + n
        else:
            if positive:
                return k562_positive.chroms(chrom)
            else:
                return k562_negative.chroms(chrom)

    train_list = []
    for index, row in h3k27ac_peaks.iterrows():    
        start = row['start'] 
        end = row['end']
        chrom = row['chr']
        mean = (start + end) // 2

        # get mean count throughout actual peak
        y_train.append(np.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end))))

        example = {
                '1kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, bound(mean, 500, True, chrom)))),
                '5kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, bound(mean, 2500, True, chrom)))),
                '10kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, bound(mean, 5000, True, chrom)))),
                '50kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, bound(mean, 25000, True, chrom)))),
                '1kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, bound(mean, 500, False, chrom)))),
                '5kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, bound(mean, 2500, False, chrom)))),
                '10kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, bound(mean, 5000, False, chrom)))),
                '50kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, bound(mean, 25000, False, chrom))))
                }
        train_list.append(example)

    bins_train = pd.DataFrame(train_list)
    print(bins_train)
    
    y_val = []
    val_list = []
    for index, row in h3k27ac_peaks_validation.iterrows():    
        start = row['start'] 
        end = row['end']
        chrom = row['chr']
        mean = (start + end) // 2

        # get mean count throughout actual peak
        y_val.append(statistics.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end))))            

        example = {
                '1kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, bound(mean, 500, True, chrom)))),
                '5kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, bound(mean, 2500, True, chrom)))),
                '10kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, bound(mean, 5000, True, chrom)))),
                '50kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, bound(mean, 25000, True, chrom)))),
                '1kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, bound(mean, 500, False, chrom)))),
                '5kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, bound(mean, 2500, False, chrom)))),
                '10kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, bound(mean, 5000, False, chrom)))),
                '50kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, bound(mean, 25000, False, chrom))))
                }
        val_list.append(example)
    
    bins_val = pd.DataFrame(val_list)
    print(bins_val)

    y_test = []
    test_list = []
    for index, row in h3k27ac_peaks_test.iterrows():    
        start = row['start'] 
        end = row['end']
        chrom = row['chr']
        mean = (start + end) // 2

        # get mean count throughout actual peak
        y_test.append(statistics.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end))))            

        example = {
                '1kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, bound(mean, 500, True, chrom)))),
                '5kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, bound(mean, 2500, True, chrom)))),
                '10kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, bound(mean, 5000, True, chrom)))),
                '50kb_positive': np.sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, bound(mean, 25000, True, chrom)))),
                '1kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, bound(mean, 500, False, chrom)))),
                '5kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, bound(mean, 2500, False, chrom)))),
                '10kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, bound(mean, 5000, False, chrom)))),
                '50kb_negative': np.sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, bound(mean, 25000, False, chrom))))
                }
        test_list.append(example)
    
    bins_test = pd.DataFrame(test_list)
    print(bins_test)
    
    return bins_train, bins_val, bins_test, y_train, y_val, y_test

bins_train, bins_val, bins_test, y_train, y_val, y_test = preprocess()
"""

In [ ]:
# TRIAL CODE
"""
import statistics
y_train = []
h3k27ac_peaks_trial = h3k27ac_peaks[:1]

def bound(mean, n, positive, chrom):
    if k562_positive.chroms(chrom) < mean + n:
        return mean + n
    else:
        if positive:
            return k562_positive.chroms(chrom)
        else:
            return k562_negative.chroms(chrom)

example_list = []
for index, row in h3k27ac_peaks_trial.iterrows():
    start = row['start']
    end = row['end']
    chrom = row['chr']
    mean = (start + end) // 2
    
    y_train.append(statistics.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end))))
    print(y_train)
    
    example = {
        '1kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, bound(mean, 500, True, chrom)))),
            '5kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, bound(mean, 2500, True, chrom)))),
            '10kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, bound(mean, 5000, True, chrom)))),
            '50kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, bound(mean, 25000, True, chrom)))),
            '1kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, bound(mean, 500, False, chrom)))),
            '5kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, bound(mean, 2500, False, chrom)))),
            '10kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, bound(mean, 5000, False, chrom)))),
            '50kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, bound(mean, 25000, False, chrom))))
    }
    print(example)
    """

In [ ]:
"""
example_list = []
example_list.append(example)
bins_train = pd.DataFrame(example_list)
print(bins_train)
"""

I tried to use numba to speed operations up but it only works on numpy and regular python expressions :(.

In [ ]:
# PREPROCESSING
# for each different peak listed as a row in h3k27ac_peak_replicated_bed, match those regions to the proseq
# positive and negative strand values
import statistics
import numba as nb

@nb.jit(nopython=True)
#def preprocess():
def preprocess(train_chr_values, train_start_values, train_end_values,
                val_chr_values, val_start_values, val_end_values,
                test_chr_values, test_start_values, test_end_values):
    y_train = []
    def bound(mean, n, positive, chrom):
        if k562_positive.chroms(chrom) < mean + n:
            return mean + n
        else:
            if positive:
                return k562_positive.chroms(chrom)
            else:
                return k562_negative.chroms(chrom)

    train_list = []
    for i in train_chr_values:
    #for index, row in h3k27ac_peaks.iterrows():    
        start = train_start_values[i] 
        end = train_end_values[i]
        chrom = train_chr_values[i]
        mean = (start + end) // 2

        # get mean count throughout actual peak
        y_train.append(statistics.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end))))

        example = {
                '1kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, bound(mean, 500, True, chrom)))),
                '5kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, bound(mean, 2500, True, chrom)))),
                '10kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, bound(mean, 5000, True, chrom)))),
                '50kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, bound(mean, 25000, True, chrom)))),
                '1kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, bound(mean, 500, False, chrom)))),
                '5kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, bound(mean, 2500, False, chrom)))),
                '10kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, bound(mean, 5000, False, chrom)))),
                '50kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, bound(mean, 25000, False, chrom))))
                }
        train_list.append(example)

    bins_train = pd.DataFrame(train_list)
    print(bins_train)
    
    y_val = []
    val_list = []
    for i in val_chr_values:    
        start = val_start_values[i]
        end = val_end_values[i]
        chrom = val_chr_values[i]
        mean = (start + end) // 2

        # get mean count throughout actual peak
        y_val.append(statistics.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end))))            

        example = {
                '1kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, bound(mean, 500, True, chrom)))),
                '5kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, bound(mean, 2500, True, chrom)))),
                '10kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, bound(mean, 5000, True, chrom)))),
                '50kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, bound(mean, 25000, True, chrom)))),
                '1kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, bound(mean, 500, False, chrom)))),
                '5kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, bound(mean, 2500, False, chrom)))),
                '10kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, bound(mean, 5000, False, chrom)))),
                '50kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, bound(mean, 25000, False, chrom))))
                }
        val_list.append(example)
    
    bins_val = pd.DataFrame(val_list)
    print(bins_val)

    y_test = []
    test_list = []
    for i in test_chr_values:    
        start = test_start_values[i] 
        end = test_end_values[i]
        chrom = test_chr_values[i]
        mean = (start + end) // 2

        # get mean count throughout actual peak
        y_test.append(statistics.mean(np.nan_to_num(h3k27ac_bigwig.values(chrom,start,end))))            

        example = {
                '1kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 500, bound(mean, 500, True, chrom)))),
                '5kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 2500, bound(mean, 2500, True, chrom)))),
                '10kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 5000, bound(mean, 5000, True, chrom)))),
                '50kb_positive': sum(np.nan_to_num(k562_positive.values(chrom, mean - 25000, bound(mean, 25000, True, chrom)))),
                '1kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 500, bound(mean, 500, False, chrom)))),
                '5kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 2500, bound(mean, 2500, False, chrom)))),
                '10kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 5000, bound(mean, 5000, False, chrom)))),
                '50kb_negative': sum(np.nan_to_num(k562_negative.values(chrom, mean - 25000, bound(mean, 25000, False, chrom))))
                }
        test_list.append(example)
    
    bins_test = pd.DataFrame(test_list)
    print(bins_test)
    
    return bins_train, bins_val, bins_test, y_train, y_val, y_test

bins_train, bins_val, bins_test, y_train, y_val, y_test = preprocess(h3k27ac_peaks['chr'].values, h3k27ac_peaks['start'].values, h3k27ac_peaks['end'].values,
                                                                    h3k27ac_peaks_val['chr'].values, h3k27ac_peaks_val['start'].values, h3k27ac_peaks_val['end'].values,
                                                                    h3k27ac_peaks_test['chr'].values, h3k27ac_peaks_test['start'].values, h3k27ac_peaks_test['end'].values)